In [1]:
import pandas as pd

def load_and_prepare_data(filepath):
    data = pd.read_csv(filepath)
    data['category'] = pd.to_datetime(data['category'].str.strip(), format='%a %b %d %Y %H%M')
    return data

def calculate_initial_average(data):
    first_30_min = data[data['category'] < data['category'].min() + pd.Timedelta(minutes=30)]
    return first_30_min['Straddle Price'].mean()

def find_threshold_and_peaks(data, average):
    threshold_30 = 1.3 * average
    above_threshold = data[data['Straddle Price'] >= threshold_30]
    peaks = above_threshold[(above_threshold['Straddle Price'] > above_threshold['Straddle Price'].shift(1)) & 
                            (above_threshold['Straddle Price'] > above_threshold['Straddle Price'].shift(-1))]
    return threshold_30, peaks

def generate_sell_signals(data, peaks):
    sell_signals = []
    for _, peak in peaks.iterrows():
        sell_price_20 = peak['Straddle Price'] * 0.8
        sell_price_70 = peak['Straddle Price'] - 70
        post_peak_data = data[data['category'] > peak['category']]
        signal_20 = post_peak_data[post_peak_data['Straddle Price'] <= sell_price_20].head(1)
        signal_70 = post_peak_data[post_peak_data['Straddle Price'] <= sell_price_70].head(1)
        if not signal_20.empty:
            sell_signals.append((peak['category'], '20% drop', signal_20.iloc[0]['category'], sell_price_20))
        if not signal_70.empty:
            sell_signals.append((peak['category'], '70 points drop', signal_70.iloc[0]['category'], sell_price_70))
    return sell_signals

def main(filepath):
    data = load_and_prepare_data(filepath)
    average_first_30 = calculate_initial_average(data)
    threshold_30, peaks = find_threshold_and_peaks(data, average_first_30)
    sell_signals = generate_sell_signals(data, peaks)
    
    print(f"Average of first 30 minutes: {average_first_30}")
    print(f"30% increase threshold: {threshold_30}")
    print("Peaks noted (time and price):")
    for index, row in peaks.iterrows():
        print(f"{row['category']} - {row['Straddle Price']}")
    print("\nSell signals (type, from peak time, signal time, target price):")
    for signal in sell_signals:
        print(f"From {signal[0]} - {signal[1]} at {signal[2]} when price reached {signal[3]}")

if __name__ == "__main__":
    main('SENSEX.csv')


Average of first 30 minutes: 423.9583333333333
30% increase threshold: 551.1458333333334
Peaks noted (time and price):
2024-09-06 09:47:00 - 560.85
2024-09-06 09:49:00 - 554.55
2024-09-06 09:52:00 - 574.4
2024-09-06 09:54:00 - 574.5

Sell signals (type, from peak time, signal time, target price):
From 2024-09-06 09:47:00 - 20% drop at 2024-09-06 10:27:00 when price reached 448.68000000000006
From 2024-09-06 09:47:00 - 70 points drop at 2024-09-06 10:03:00 when price reached 490.85
From 2024-09-06 09:49:00 - 20% drop at 2024-09-06 10:27:00 when price reached 443.64
From 2024-09-06 09:49:00 - 70 points drop at 2024-09-06 10:04:00 when price reached 484.54999999999995
From 2024-09-06 09:52:00 - 20% drop at 2024-09-06 10:25:00 when price reached 459.52
From 2024-09-06 09:52:00 - 70 points drop at 2024-09-06 10:02:00 when price reached 504.4
From 2024-09-06 09:54:00 - 20% drop at 2024-09-06 10:25:00 when price reached 459.6
From 2024-09-06 09:54:00 - 70 points drop at 2024-09-06 10:02:00 wh